In [1]:
%config IPCompleter.greedy=True
import pickle
import json
from tqdm import tqdm_notebook as tqdm
from deeppavlov import configs, build_model

In [2]:
def json_read(filename):
    with open(filename, 'r') as inf:
        res = json.load(inf)
    return res

def json_dump(obj, filename, ea=False, indent=4):
    with open(filename, 'w') as ouf:
        json.dump(obj, ouf, ensure_ascii=ea, indent=indent)

### Initializing and checking model

In [3]:
kbqa_model = build_model(configs.kbqa.kbqa_rus)

2020-03-28 14:24:13.154 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /Volumes/Seagate/conda/.deeppavlov/models/kbqa_mix_lowercase/relation_prediction/classes.dict]
[nltk_data] Downloading package punkt to /Users/ne0n/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/ne0n/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /Users/ne0n/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /Users/ne0n/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!
2020-03-28 14:24:15.49 INFO in 'deeppavlov.models.embedders.fasttext_embedder'['fasttext_embedder'] at line 67: [loading fastText embeddings from `/Volumes/Seagate/conda/.deeppavlov/downloads/embeddings/ft_native_300_ru_wiki_lenta_

2020-03-28 14:25:04.311 INFO in 'deeppavlov.models.classifiers.keras_classification_model'['keras_classification_model'] at line 273: [initializing `KerasClassificationModel` from saved]


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


2020-03-28 14:25:05.285 INFO in 'deeppavlov.models.classifiers.keras_classification_model'['keras_classification_model'] at line 283: [loading weights from model.h5]


2020-03-28 14:25:05.824 INFO in 'deeppavlov.models.classifiers.keras_classification_model'['keras_classification_model'] at line 134: Model was successfully initialized!
Model summary:
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 36, 300)      0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 36, 256)      77056       input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 36, 256)      153856      input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_3 (Conv1D)      

...

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
seq_dim is deprecated, use seq_axis instead
Instructions for updating:
batch_dim is deprecated, use batch_axis instead


2020-03-28 14:25:19.968 INFO in 'deeppavlov.core.layers.tf_layers'['tf_layers'] at line 756: 
Warning! tf.contrib.cudnn_rnn.CudnnCompatibleLSTMCell is used. It is okay for inference mode, but if you train your model with this cell it could NOT be used with tf.contrib.cudnn_rnn.CudnnLSTMCell later. 


Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use standard file APIs to check for files with this prefix.


2020-03-28 14:25:24.650 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /Volumes/Seagate/conda/.deeppavlov/models/kbqa_mix_lowercase/ner/model]



INFO:tensorflow:Restoring parameters from /Volumes/Seagate/conda/.deeppavlov/models/kbqa_mix_lowercase/ner/model


In [4]:
kbqa_model(['В какой стране находится город Гагарин?'])

['Китайская Народная Республика']

### Getting answers from deeppavlov model

In [5]:
dataset = json_read('kbqa_russian_dataset.json')

In [6]:
deeppavlov_answers = {}

In [7]:
for i, entry in enumerate(tqdm(dataset[1178:1200])):
    uid = entry['uid']
    q = entry['question_text']
    print('\r' + ' ' * 500 + f'\r{q}', end='')
    dp_ans = kbqa_model([q])
    deeppavlov_answers[uid] = dp_ans
    if i % 50 == 0:
        json_dump(deeppavlov_answers, 'deeppavlov_answers_new22.json')
json_dump(deeppavlov_answers, 'deeppavlov_answers_new22.json')

Когда началась первая мировая война?                                                                                                                                                                                                                                                                                                                                                                                                                                                                                


### Testing on dataset

In [3]:
dataset = json_read('kbqa_russian_dataset.json')

In [5]:
deeppavlov_answers = json_read('deeppavlov_answers.json')

In [6]:
with open('/Volumes/Seagate/conda/.deeppavlov/downloads/wikidata_rus/wiki_rus_q_to_name.pickle', 'rb') as inf:
    q_to_name = pickle.load(inf)

In [39]:
def get_ans_names(act_anses):
    act_anses_q = [ans['value'].split('/')[-1] for ans in act_anses]
    act_anses = []
    for act_ans_q in act_anses_q:
        if act_ans_q in q_to_name:
            act_anses.append(q_to_name[act_ans_q]['name'])
        else:
            act_anses.append(act_ans_q)
    return act_anses

In [40]:
def get_eval_result(act_anses, dp_anses):
    if dp_anses[0] == 'Not Found':
        return 'not found'
    
    for ans in act_anses:
        if ans['type'] != 'uri':
            return 'manual'
        
    act_anses = get_ans_names(act_anses)
    
    if len(act_anses) != len(dp_anses):
        for dp_ans in dp_anses:
            if dp_ans not in act_anses:
                return 'wrong'
        return 'contains'
    
    for act_ans, dp_ans in zip(act_anses, dp_anses):
        if act_ans != dp_ans:
            return 'wrong'
        
    return 'correct'

In [44]:
cnt = {
    'correct': 0,
    'contains': 0,
    'not found': 0,
    'wrong': 0,
    'manual': 0
}
eval_res_json = {}
with open('deeppavlov_evaluation_result_log.txt', 'w') as fres, \
     open('deeppavlov_evaluation_manual.txt', 'w') as fman:
    for entry in dataset:
        uid = str(entry['uid'])
        dp_anses = sorted(deeppavlov_answers[uid])
        
        eval_result = get_eval_result(entry['answers'], dp_anses)
        act_anses = get_ans_names(entry['answers'])
        
        if eval_result == 'manual':
            print(uid, file=fman)
            print(f'Dataset: {act_anses}', file=fman)
            print(f'deeppavlov: {deeppavlov_answers[uid]}', file=fman)
            print(file=fman)
            print(f'uid: {uid}; manual', file=fres)
        
        if eval_result == 'wrong':
            print(f'uid: {uid}; wrong; actual: {act_anses}; dp: {dp_anses}', file=fres)
            
        if eval_result == 'contains':
            print(f'uid: {uid}; contains in; actual: {act_anses}; dp: {dp_anses}', file=fres)
            
        if eval_result == 'correct':
            print(f'uid: {uid}; correct; actual: {act_anses}', file=fres)
            
        cnt[eval_result] += 1
        eval_res_json[int(uid)] = eval_result
        
    print(cnt, file=fres)
    
json_dump(eval_res_json, 'deeppavlov_evaluation_result.json')

#### Results by tagset

In [8]:
tagset = json_read('tagset.json')

In [4]:
dataset = json_read('../dataset compilation/kbqa_russian_dataset.json')
test_uids = set(json_read('../dataset compilation/test_uids.json'))
dev_uids = set(json_read('../dataset compilation/dev_uids.json'))
tagset = set(json_read('../dataset compilation/tagset.json'))
eval_results = json_read('deeppavlov_evaluation_result.json')

In [10]:
tagwise_results = {}
for tag in tagset:
    tagwise_results[tag] = {
        'correct': 0,
        'contains': 0,
        'not found': 0,
        'wrong': 0,
        'manual': 0
    }
tagwise_results['overall'] = {
        'correct': 0,
        'contains': 0,
        'not found': 0,
        'wrong': 0,
        'manual': 0
    }

In [11]:
for entry in dataset[:1200]:
    uid = entry['uid']
    if uid not in test_uids:
        continue
    uid = str(uid)
    tags = entry['tags'] + ['overall']
    er = eval_results[uid]
    for tag in tags:
        tagwise_results[tag][er] += 1
json_dump(tagwise_results, 'tagwise_results_on_test.json')

In [33]:
results_qna = json_read('deeppavlov_answers_qna.json')

In [47]:
cnt = 0
for uid, ans in results_qna.items():
    if int(uid) not in test_uids:
        continue
    if ans[0] == 'Not Found':
        cnt += 1
cnt

175

In [6]:
for entry in dataset[:1200]:
    uid = entry['uid']
    if uid not in test_uids:
        continue
    uid = str(uid)
    tags = entry['tags'] + ['overall']
    er = eval_results[uid]
    if (er == 'correct' or er == 'contains') and '1-hop' not in tags:
        print(entry)

{'uid': 186, 'question_text': 'На какой реке стоит город Прага?', 'query': 'SELECT ?answer \nWHERE {\n  wd:Q1085 wdt:P206 ?answer .\n  ?answer wdt:P31 wd:Q4022\n}', 'answer_text': 'Влтава', 'question_uris': ['http://www.wikidata.org/entity/Q1085', 'http://www.wikidata.org/entity/Q4022'], 'question_props': ['P206', 'P31'], 'answers': [{'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q131574'}], 'tags': ['multi-constraint'], 'question_eng': 'What river is the city of Prague on?'}
{'uid': 561, 'question_text': 'Капитаном какой московской футбольной команды был Игорь Нетто?', 'query': 'SELECT ?answer \nWHERE {\n  wd:Q48355 wdt:P54 ?answer .\n  ?answer wdt:P31 wd:Q476028 .\n  ?answer wdt:P159 wd:Q649 .\n}', 'answer_text': 'Спартак', 'question_uris': ['http://www.wikidata.org/entity/Q649', 'http://www.wikidata.org/entity/Q48355', 'http://www.wikidata.org/entity/Q476028'], 'question_props': ['P159', 'P31', 'P54'], 'answers': [{'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q2

### Exploration of embedded data

In [2]:
with open('/Volumes/Seagate/conda/.deeppavlov/downloads/wikidata_rus/wiki_rus.pickle', 'rb') as inf:
    obj = pickle.load(inf)

In [4]:
len(obj)

4634023

In [5]:
for i, (k, v) in enumerate(obj.items()):
    print(k, v)
    if i == 1:
        break

Q31 [['P530', 'Q32', 'Q38', 'Q183', 'Q347', 'Q408', 'Q212', 'Q1246', 'Q142', 'Q145', 'Q16', 'Q252', 'Q35', 'Q43', 'Q55', 'Q77', 'Q833', 'Q843', 'Q865', 'Q96', 'Q974', 'Q30', 'Q159', 'Q668', 'Q41', 'Q148', 'Q230', 'Q801'], ['P1464', 'Q7463296'], ['P1465', 'Q8366417'], ['P2238', 'Q130201'], ['P138', 'Q206443'], ['P78', 'Q39773'], ['P1344', 'Q1088364'], ['P910', 'Q4366768'], ['P5125', 'Q7112200'], ['P610', 'Q322824'], ['P208', 'Q390947'], ['P209', 'Q1755321', 'Q1230309', 'Q1761425'], ['P122', 'Q41614', 'Q3330103'], ['P85', 'Q161539'], ['P1546', 'Q1308013', 'Q20760289'], ['P1792', 'Q7021332'], ['P2852', 'Q1061257', 'Q25648793', 'Q25648794'], ['P1791', 'Q7974978'], ['P1151', 'Q3247091'], ['P194', 'Q1137059'], ['P92', 'Q633629'], ['P1552', 'Q3174312'], ['P17', 'Q31'], ['P6', 'Q950958'], ['P2633', 'Q1115035'], ['P463', 'Q458', 'Q1065', 'Q7184', 'Q41550', 'Q8908', 'Q13116', 'Q42262', 'Q7825', 'Q141720', 'Q1542735', 'Q152299', 'Q151991', 'Q81299', 'Q191384', 'Q827525', 'Q656801', 'Q1043527', 'Q

In [7]:
obj['Q134318']

[['P1464', 'Q23693271'],
 ['P1376', 'Q1983158', 'Q28589963'],
 ['P1792', 'Q32755035'],
 ['P190', 'Q3791'],
 ['P1791', 'Q32236801'],
 ['P131', 'Q1983158'],
 ['P138', 'Q7327'],
 ['P1465', 'Q24100574'],
 ['P31', 'Q515', 'Q192287'],
 ['P910', 'Q22913477'],
 ['P17', 'Q159', 'Q15180', 'Q34266', 'Q2305208', 'Q139319'],
 ['P1343', 'Q4114391', 'Q602358', 'Q4173137', 'Q19180675'],
 ['P163', 'Q4485895'],
 ['P2046', '14.46'],
 ['P0', 'город в России'],
 ['P2046', '14.46']]

In [4]:
with open('/Volumes/Seagate/conda/.deeppavlov/downloads/wikidata_rus/templates.pickle', 'rb') as inf:
    temps = pickle.load(inf)

In [5]:
temps

{'Какой художественный стиль XXX': 'P149',
 'Художественный стиль XXX  это что': 'P149',
 'В каком стиле XXX': 'P149',
 'В какой сети появился XXX': 'P449',
 'Какая телекомпания выпустила XXX': 'P449',
 'По какому каналу идет XXX': 'P449',
 'Какой телеканал впервые показал XXX': 'P449',
 'На каком канале показывают XXX': 'P449',
 'По какому каналу впервые показали XXX': 'P449',
 'На каком канале впервые появился XXX': 'P449',
 'Какой канал транслировал XXX': 'P449',
 'На каком телеканале появился XXX': 'P449',
 'В эфире какой станции вышел XXX': 'P449',
 'На какой радиостанции была премьера XXX': 'P449',
 'По какому каналу впервые начали крутить XXX': 'P449',
 'По какому телевизионному каналу впревые шло XXX': 'P449',
 'Шоу XXX изначально транслировалось по какой сети': 'P449',
 'Какая сеть транслировала XXX': 'P449',
 'XXX в эфире какого канала': 'P449',
 'По какому каналу XXX впервые показывали': 'P449',
 'Какой канал показывает XXX': 'P449',
 'С каким телеканалом связан XXX': 'P449'

In [7]:
len(set(temps.values()))

142